In [1]:
# Anamorphic Encryption - ElGamal Scheme
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from char_int_elgamal import encode_msg_to_int, decode_msg_to_int
import secrets
import math
from itertools import zip_longest

<h1> ElGamal: Baby step - Giant step </h1>

In [ ]:
# ======== HELPER FUNCTIONS ======== #


# Pseudo-random function
def F(sk_a, IV, st, q):
    prg_input = st.to_bytes(16, 'little')  # AES block size = 16 bytes
    iv_bytes = IV.to_bytes(16, 'little')
    
    aes = AES.new(sk_a, AES.MODE_CBC, iv=iv_bytes)
    encrypted = aes.encrypt(prg_input)
    
    r_prime = (int.from_bytes(encrypted, 'little') % (q - 1)) + 1
    return r_prime

def g_generator(p):
    while True:
        h = secrets.randbelow(p-3) + 2
        g = pow(h, 2, p)
        if g != 1:
            return g


# ======== KEY GENERATION ======== #


# Standard public/private + double key generation
def create_keys(p, q, g):
    sk_a = get_random_bytes(16) # double key
    sk = secrets.randbelow(q) # secret key
    pk = pow(g, sk, p) # public key
    return pk, sk, sk_a


# ======== ANAMORPHIC ENCRYPTION ======== #


def aEncrypt(p, q, g, m, m_a, pk, sk_a, IV, st):
    # Generate pseudo-random r' to compute r
    r_prime = F(sk_a, IV, st, q)
    r = (r_prime + m_a) % q

    # Calculate ElGamal ciphertext
    pk_r = pow(pk, r, p)
    ct0 = (m * pk_r) % p
    ct1 = pow(g, r, p)
    ct = (ct0, ct1)

    # Increment state for next message
    st += 1

    return ct, st


def baby_step_giant_step(p, g, h, bound):
    if h == 1:
        return 0  # g^0 == 1
    
    # Limit bound to at least 1
    if bound <= 1:
        return None

    # Block size n ≈ sqrt(bound) for BSGS
    n = math.isqrt(bound)
    if n * n < bound:
        n += 1

    # Baby-steps: store g^j -> for all j in [0, n-1]
    baby = {}
    current = 1
    for j in range(n):
        # If collision, keep the smallest j
        if current not in baby:
            baby[current] = j
        current = (current * g) % p

    # Compute g^n
    g_n = pow(g, n, p)
    # Modular inverse of g_n modulo p
    g_n_inv = pow(g_n, p - 2, p)

    # Giant-steps: look for i, j s.t. x = i * n + j solves g^x = h
    gamma = h
    max_i = (bound + n - 1) // n  # Compute how many giant steps we need

    for i in range(max_i):
        # Check if current gamma matches any baby table value 
        if gamma in baby:
            j = baby[gamma]     # Retrieve corresponding baby-step exponent j
            x = i * n + j       # Reconstruct the full exponent x = i*n + j
            if x < bound:       # Only accept x if it is within the allowed bound
                return x
            else:
                return None
        # Move to next giant step -> multiply gamma by g^-n
        gamma = (gamma * g_n_inv) % p

    return None


# Anamorphic decryption
def aDecrypt(p, g, sk_a, ct, IV, st, q, bound):
    r_prime = F(sk_a, IV, st, q)

    # compute inverse of g^r_prime
    g_rprime = pow(g, r_prime, p)
    g_rprime_inv = pow(g_rprime, p - 2, p)
    g_m_a = (ct[1] * g_rprime_inv) % p

    m_a = baby_step_giant_step(p, g, g_m_a, bound)
    return m_a


# Standard decryption
def Decrypt(p, ct, sk):
    s = pow(ct[1], sk, p) # s = g^r^x = y^r
    s_inverse = pow(s, -1, p)
    cover_msg = (ct[0] * s_inverse) % p
    return cover_msg


# ===== Parameter initialization ===== #

p = 25283138329189278652587895589109525736072750946542698825287111445816073512149787631506175333955884685211183346377467560941062660497423325529940869143458703 
q = 12641569164594639326293947794554762868036375473271349412643555722908036756074893815753087666977942342605591673188733780470531330248711662764970434571729351
g = g_generator(p)

pk, sk, sk_a = create_keys(p, q, g)

st = 0
IV = 0

bound = 2**36

In [ ]:
# ===== Int encryption test ===== #

m = 13000
m_a = 200000005
print(f'm = {m}, m_a = {m_a}')

ct, st_new = aEncrypt(p, q, g, m, m_a, pk, sk_a, IV, st)
print(f"Ciphertext: {ct}")


# ===== Int decryption test ===== #

cover_m = Decrypt(p, ct, sk)
secret_m = aDecrypt(p, g, sk_a, ct, IV, st, q, bound)
print(f'cover_m = {cover_m}, secret_m = {secret_m}')

m = 13000, m_a = 200000005
Ciphertext: (1913861343883343165279614105683747021945195540403198509746410286484321452452719416964902110752770525867769768353015993528226601430045082181132350274213853, 16004849847820936426070952803722593183942085963746635600526831786933875850736605476908309196631562944155237871097296556031948759547397099209990429891344214)
cover_m = 13000, secret_m = 200000005


In [ ]:
# ===== Message encryption test ===== #

def encode_msg(m: str, m_a: str):
    # Convert each word to integers
    int_array = [encode_msg_to_int(word) for word in m.split()]
    a_int_array = [encode_msg_to_int(word_a) for word_a in m_a.split()]

    pairs = zip_longest(int_array, a_int_array, fillvalue=0)

    cts = []
    st_local = st
    for m_int, m_a_int in pairs:
        ct, st_local = aEncrypt(p, q, g, m_int, m_a_int, pk, sk_a, IV, st_local)
        cts.append(ct)

    return cts, st_local

def decode_msg(cts, st_start):
    decoded_cover = ""
    decoded_anam = ""
    st_local = st_start

    for ct in cts:
        m = Decrypt(p, ct, sk)
        m_a = aDecrypt(p, g, sk_a, ct, IV, st_local, q, bound)
        decoded_cover += decode_msg_to_int(m) + " "
        decoded_anam += (decode_msg_to_int(m_a) if m_a is not None else "[?]") + " "
        st_local += 1

    return decoded_cover.strip(), decoded_anam.strip()


m = "Hi, how are you doing James?"
m_a = "We attack at dawn, my friend" # Only len(words) <= 6 works

# Encode to ciphertext
cts, st_after = encode_msg(m, m_a)
print("Ciphertext: ", cts)

# Decode to text
decoded_m, decoded_m_a = decode_msg(cts, st)
print(f'cover_m: {decoded_m} \nsecret_m: {decoded_m_a}')

Ciphertext:  [(20295020438589246510404717175266614248816082207327169229260959157461671622531895819822012492495430582007819818569682308864916002590391699215961158891473850, 21265265556760049026052493860097893894458533087614878555845307087666329064007627996606936508729950765673470564549018879262899530965229806598208451250006510), (12789187275919634986461611456159756454834351055135444470523279323768639030123749034247996453211413902696802883697575737131350449848445662918225898767206677, 6201156515671953617889311545798760798657634763309888083392716645657319268872955853900235146265514678845010668166210933541529890314410392801561976569709784), (7232464662942471798584878158481261967718991566871828877755436701386005983823913206722833143678360913449763287833806841690942292168826261614759875835175017, 13743579212917463436475711023410329371393891086832458129155409280891065360384407365569110681670072881021573648092386364292357795884529116739706739019970439), (115383731406662890725035694330106509155

### To implement:
* ElGamal ✅
* Baby step / giant step ✅
* Implement static strong prime ✅
* Encrypt messages instead of integers ✅
* Write down progress + knowledge in Overleaf ✅
* Read research paper 🛑
* Add new ElGamal version to PM 🛑
* Implement password protection + panic password 🛑
* Final UI + ai chatbot 🛑
* Write full Overleaf paper 🛑

In [13]:
import sys
import libnum

bitsize=2048

if (len(sys.argv)>1):
        bitsize=int(sys.argv[1])

r=libnum.randint_bits(bitsize)

print ("Random: %d Length: %d" % (r,libnum.len_in_bits(r)))

p=libnum.generate_prime(bitsize)

print ("\nPrime (p): %d. Length: %d bits, Digits: %d" % (p,libnum.len_in_bits(p), len(str(p)) ))  


q=libnum.generate_prime(bitsize)
print ("\nPrime (q): %d. Length: %d bits, Digits: %d" % (q,libnum.len_in_bits(q), len(str(q)) ))

N=p*q
print ("\nPrime (N): %d. Length: %d bits, Digits: %d" % (N,libnum.len_in_bits(N), len(str(N)) ))

ValueError: invalid literal for int() with base 10: '--f=/Users/maverick/Library/Jupyter/runtime/kernel-v32788d0436e168ad6d2061bc3fb0a00be6e063992.json'